In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import product
from sklearn.preprocessing import LabelEncoder

import time
import datetime



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#データの読み込み
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
cats = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test  = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
train

In [ ]:
train[train.item_cnt_day<0]

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
flierprops = dict(marker='^', markerfacecolor='purple', ms=12,
                  linestyle='none', markeredgecolor='black')
sns.boxplot(x=train.item_cnt_day, flierprops=flierprops)

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price, flierprops=flierprops)

In [ ]:
# 図から外れ値を除外
train = train[(train.item_price < 300000 )& (train.item_cnt_day < 1000)]
# item_priceが0以下のものを除外
# reset_indexはしとくと良いかも
train = train[train.item_price > 0].reset_index(drop = True)
# 返品処理されたもの(値がマイナス)は、売上0として扱う
# train.loc[train.item_cnt_day < 1, "item_cnt_day"] = 0
train

In [ ]:
display(train[train.item_cnt_day<0])
display(train[train.item_price<=0])

In [ ]:
display(shops[shops.shop_name.duplicated()])
# nameの被りなしだが, 誤植などは存在する
shops.shop_name

In [ ]:
display(train[train.shop_id==0].sort_values("date_block_num"))
display(train[train.shop_id==57].sort_values("date_block_num"))

In [ ]:
display(train[train.shop_id==1].sort_values("date_block_num"))
display(train[train.shop_id==58].sort_values("date_block_num"))

In [ ]:
display(train[(train.shop_id==10) & (train.date_block_num==25)].sort_values("date_block_num"))
display(train[train.shop_id==11].sort_values("date_block_num"))

In [ ]:
shop_id_month_list = []
for i in range(0,60):
    unq = train[train.shop_id==i].date_block_num.unique()
    #print(f"{i} {train[train.shop_id==i].date_block_num.unique()}")
    for j in unq:
       shop_id_month_list.append([i, j])
plt.figure(figsize = (12,8))
x, y = zip(*shop_id_month_list)
sns.scatterplot(x,y, s=50)
plt.show()

In [ ]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [ ]:
shops.loc[ shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"'
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] ) # city追加
shops["category"] = shops.shop_name.str.split(" ").map( lambda x: x[1] ) # category 追加
shops.loc[shops.city == "!Якутск", "city"] = "Якутск"
shops.head()

In [ ]:
shops["category"].value_counts()

In [ ]:
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5:
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )

In [ ]:
shops["shop_category"] = LabelEncoder().fit_transform( shops.category )
shops["shop_city"] = LabelEncoder().fit_transform( shops.city )
# display(shops.head())
shops = shops[["shop_id", "shop_category", "shop_city"]]
display(shops.head())

In [ ]:
cats.head()

In [ ]:
"""
cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
# cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
cats.loc[ (cats.type_code == "Игровые")| (cats.type_code == "Аксессуары"), "type_code" ] = "Игры"
cats.head()
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
# cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
"""

In [ ]:
cats["type_code"] = cats.item_category_name.apply( lambda x: x.split(" ")[0] ).astype(str)
cats.loc[ (cats.type_code == "Игровые")| (cats.type_code == "Аксессуары"), "type_code" ] = "Игры"
cats.head()

In [ ]:
cats["type_code"].value_counts()

In [ ]:
category = []
for cat in cats.type_code.unique():
    if len(cats[cats.type_code == cat]) >= 5: 
        category.append( cat )
cats.type_code = cats.type_code.apply(lambda x: x if (x in category) else "etc")

In [ ]:
cats.type_code.value_counts()

In [ ]:
cats.type_code = LabelEncoder().fit_transform(cats.type_code)
cats["split"] = cats.item_category_name.apply(lambda x: x.split("-"))
cats["subtype"] = cats.split.apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats["subtype_code"] = LabelEncoder().fit_transform( cats["subtype"] )
display(cats.head())
cats = cats[["item_category_id", "subtype_code", "type_code"]]
display(cats.head())

In [ ]:
items

In [ ]:
items[items.item_name.duplicated()]

In [ ]:
import re
def name_correction(x):
    x = x.lower() # all letters lower case
    x = x.partition('[')[0] # partition by square brackets
    x = x.partition('(')[0] # partition by curly brackets
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) # remove special characters
    x = x.replace('  ', ' ') # replace double spaces with single spaces
    x = x.strip() # remove leading and trailing white space
    return x

In [ ]:
items.item_name.unique()

In [ ]:
items.item_name.isna().sum()

In [ ]:
name_correction("PC, Цифровая версия]")

In [ ]:
tmp = "AidfhoasgDFhs oghas Su!!["
re.sub("[^A-Z]+", " ", tmp).lower()

In [ ]:
pd.Series(["!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]"]).str.replace('[^A-Za-z0-9А-Яа-я]+', " ", regex=False).str.lower().values


In [ ]:
#tmp1,tmp2=
pd.set_option("display.max_colwidth", 80)
tmp=pd.Series(["!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]"]).str.split("[", 1)
display(tmp)
#tmp2

In [ ]:
pd.Series(["!ABBYY FineReader 12 Professional Edition Full [PC, Цифровая версия]"]).str.replace('[^A-Za-z0-9А-Яа-я]+', " ", regex=True).str.lower().values

In [ ]:
tmp = lambda x: x[:-1] if x !="0" else "0"
tmp("pc цифровая версия")

In [ ]:
tmp = "aistabetai"
tmp[:-1]

In [ ]:
# Clean item names.

# split item names by first bracket
# [または(で前後に分けている
items["name1"], items["name2"] = items.item_name.str.split("[", 1).str
items["name1"], items["name3"] = items.item_name.str.split("(", 1).str
display(items.head(10))

# replace special characters and turn to lower case
items["name2"] = items.name2.str.replace('[^A-Za-z0-9А-Яа-я]+', " ", regex=True).str.lower() # 括弧とかが削除されている
items["name3"] = items.name3.str.replace('[^A-Za-z0-9А-Яа-я]+', " ", regex=True).str.lower()
display(items.head(10))

# fill nulls with '0'
items = items.fillna('0')
display(items.head(10))

items["item_name"] = items["item_name"].apply(lambda x: name_correction(x))
display(items.head(10))

# return all characters except the last if name 2 is not "0" - the closing bracket
items.name2 = items.name2.apply( lambda x: x[:-1] if x !="0" else "0")
display(items.head(10))


In [ ]:
items[(items.name3=="0") & (items.name2=="0")]

In [ ]:
items.name2.value_counts()

In [ ]:
items["type"] = items.name2.apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0] )
items[(items.type == "x360") | (items.type == "xbox360")]

In [ ]:
items[items.name2==""]

In [ ]:
 items[(items.type == 'pc' )| (items.type == 'pс') | (items.type == "pc")]

In [ ]:
items[(items.type == "pc")]
# 同じ書き方
# items[(items.type =='pc')]

In [ ]:
'pс'.encode()

In [ ]:
items.type.value_counts()

In [ ]:
"  sa".split(" ")

In [ ]:
'рs3'.encode()

In [ ]:
"ps3".encode()

In [ ]:
tmp=items["type"].apply(lambda x:x.encode())
tmp.value_counts()

In [ ]:
items["type"] = items.name2.apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0] )
items.loc[(items.type == "x360") | (items.type == "xbox360") | (items.type == "xbox 360") ,"type"] = "xbox 360"
items.loc[ items.type == "", "type"] = "mac"
items.type = items.type.apply( lambda x: x.replace(" ", "") )
items.loc[ (items.type == "рс")| (items.type == 'pс') | (items.type == "pc"), "type" ] = "pc"
items.loc[ items.type == 'рs3' , "type"] = "ps3"

In [ ]:
tmp=items["type"].apply(lambda x:x.encode())
tmp.value_counts()

In [ ]:
items.type.value_counts()

In [ ]:
items

In [ ]:
items.groupby(["type"]).agg({"item_id": "count"})

In [ ]:
tmp=items.groupby(["type"]).agg({"item_id": "count"})
tmp = tmp.reset_index()
tmp.loc[(tmp.type == "0"), "item_id"].values[0]

In [ ]:
items.name2.value_counts()

In [ ]:
items.type.value_counts()

In [ ]:
group_sum = items.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()

display(items[items.name2=="other"])

for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] <40:
        # drop_cols.append(cat)
        items.loc[(items["type"] == cat), "name2"] = "other"
        items.loc[(items["type"] == cat), "type"] = "other"
        
# items = items.drop(["type"], axis = 1)
display(items[items.name2=="other"])
display(items[items.type=="other"])

display(items.head(10))

In [ ]:
items[items.name2=="other"].shape

In [ ]:
items.type.value_counts()

In [ ]:
items["name2"].value_counts()

In [ ]:
items.name2 = LabelEncoder().fit_transform(items.name2)
items.name3 = LabelEncoder().fit_transform(items.name3)

items.type = LabelEncoder().fit_transform(items.type)

items.drop(["item_name", "name1"],axis = 1, inplace= True)
items.head()

## 組み合わせ

In [ ]:
ts = time.time()
matrix = []
cols  = ["date_block_num", "shop_id", "item_id"]
for i in range(34):
    sales = train[train.date_block_num == i]
    matrix.append( np.array(list( product( [i], sales.shop_id.unique(), sales.item_id.unique() ) ), dtype = np.int16) )

matrix = pd.DataFrame( np.vstack(matrix), columns = cols )
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values( cols, inplace = True )
time.time()- ts

In [ ]:
matrix.isna().sum()

In [ ]:
matrix

In [ ]:
# add revenue to train df
train["revenue"] = train["item_cnt_day"] * train["item_price"]
train

In [ ]:
train.revenue.describe()

In [ ]:
matrix[matrix.duplicated()]

In [ ]:
ts = time.time()
group = train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
group.columns = ["item_cnt_month"]
group.reset_index( inplace = True)
matrix = pd.merge( matrix, group, on = cols, how = "left" )
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).astype(np.float16)
time.time() - ts

In [ ]:
matrix.isna().sum()

In [ ]:
matrix

In [ ]:
test

In [ ]:
test["date_block_num"] = 34
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test["shop_id"] = test.shop_id.astype(np.int8)
test["item_id"] = test.item_id.astype(np.int16)

In [ ]:
matrix.isna().sum()

In [ ]:
test.drop(["ID"],axis = 1)

In [ ]:
ts = time.time()
# cols  = ["date_block_num", "shop_id", "item_id"]
matrix = pd.concat([matrix, test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=cols)
matrix.fillna( 0, inplace = True )
time.time() - ts

In [ ]:
matrix.isna().sum()

In [ ]:
print(f"shops:{shops.columns}")
print(f"items:{items.columns}")
print(f"cats:{cats.columns}")
print(f"matrix:{matrix.columns}")

In [ ]:
ts = time.time()
matrix = pd.merge( matrix, shops, on = ["shop_id"], how = "left" )
matrix = pd.merge(matrix, items, on = ["item_id"], how = "left")
matrix = pd.merge( matrix, cats, on = ["item_category_id"], how = "left" )
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["item_category_id"] = matrix["item_category_id"].astype(np.int8)
matrix["subtype_code"] = matrix["subtype_code"].astype(np.int8)
matrix["name2"] = matrix["name2"].astype(np.int8)
matrix["name3"] = matrix["name3"].astype(np.int16)
matrix["type_code"] = matrix["type_code"].astype(np.int8)
time.time() - ts

In [ ]:
matrix.isna().sum()

In [ ]:
matrix

In [ ]:
# Define a lag feature function
def lag_feature( df,lags, cols ):
    for col in cols:
        print(col)
        tmp = df[["date_block_num", "shop_id","item_id",col ]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
ts = time.time()
matrix = lag_feature( matrix, [1,2,3], ["item_cnt_month"] )
time.time() - ts

In [ ]:
matrix.isna().sum()

In [ ]:
ts = time.time()
group = matrix.groupby( ["date_block_num"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num"], how = "left")
matrix.date_avg_item_cnt = matrix["date_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1], ["date_avg_item_cnt"] )
matrix.drop( ["date_avg_item_cnt"], axis = 1, inplace = True )
time.time() - ts

In [ ]:
matrix.isna().sum()

In [ ]:
group

In [ ]:
ts = time.time()
group = train.groupby( ["date_block_num","shop_id"] ).agg({"revenue": ["sum"] })
group.columns = ["date_shop_revenue"]
group.reset_index(inplace = True)

matrix = matrix.merge( group , on = ["date_block_num", "shop_id"], how = "left" )
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)
time.time() - ts

In [ ]:
group.head()

In [ ]:
matrix

In [ ]:
ts = time.time()
group = group.groupby(["shop_id"]).agg({ "date_shop_revenue":["mean"] })
group.columns = ["shop_avg_revenue"]
group.reset_index(inplace = True )

matrix = matrix.merge( group, on = ["shop_id"], how = "left" )
matrix["shop_avg_revenue"] = matrix.shop_avg_revenue.astype(np.float32)
matrix["delta_revenue"] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix["delta_revenue"] = matrix["delta_revenue"]. astype(np.float32)

matrix = lag_feature(matrix, [1], ["delta_revenue"])
matrix["delta_revenue_lag_1"] = matrix["delta_revenue_lag_1"].astype(np.float32)
matrix.drop( ["date_shop_revenue", "shop_avg_revenue", "delta_revenue"] ,axis = 1, inplace = True)
time.time() - ts

In [ ]:
matrix["month"] = (matrix["date_block_num"]%12)+1
matrix["year"] = (matrix["date_block_num"]//12)+2013

In [ ]:
matrix

In [ ]:
 matrix["date_block_num"] - matrix.groupby(["item_id","shop_id"])["date_block_num"].transform('min')

In [ ]:
 matrix.groupby(["item_id","shop_id"])["date_block_num"].transform('min')

In [ ]:
 matrix.groupby(["item_id","shop_id"]).agg({"date_block_num":"min"})

In [ ]:
ts = time.time()
matrix["item_shop_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id","shop_id"])["date_block_num"].transform('min')
matrix["item_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id"])["date_block_num"].transform('min')
time.time() - ts

In [ ]:
matrix

In [ ]:
ts = time.time()
matrix = matrix[matrix["date_block_num"] >= 3]
time.time() - ts

In [ ]:
matrix.isna().sum()

In [ ]:
#for i in range(35):
#   print(f"{i}:{matrix[matrix.date_block_num==i].isnull().sum()}")

In [ ]:
matrix.info()

In [ ]:
matrix.head().T

## モデリング

In [ ]:
import gc
data = matrix.copy()
# del matrix
gc.collect()

In [ ]:
data[data["date_block_num"]==34].shape

In [ ]:
data

In [ ]:
train_X = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
train_y = data[data.date_block_num < 33]['item_cnt_month']
val_X = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
val_y = data[data.date_block_num == 33]['item_cnt_month']
test_X = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
del data
gc.collect()

In [ ]:
train_y = train_y.clip(0, 20)
val_y = val_y.clip(0, 20)

In [ ]:
from optuna.integration import lightgbm as lgb
from lightgbm import plot_importance
import time

In [ ]:
ts = time.time()

dtrain = lgb.Dataset(train_X, label=train_y)
eval_data = lgb.Dataset(val_X, label=val_y)

param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
    }

best = lgb.train(param, 
                 dtrain,
                 valid_sets=eval_data,
                 early_stopping_rounds=50)

time.time() - ts

In [ ]:
print(best.params)
print(best.best_iteration)
print(best.best_score)

In [ ]:
test_y = best.predict(test_X).clip(0, 20)
test_X['item_cnt_month'] = test_y
submission = pd.merge(
    test,
    test_X[['shop_id','item_id','item_cnt_month']],
    on=['shop_id','item_id'],
    how='left'
)
# 提出ファイル作成
submission[['ID','item_cnt_month']].to_csv('submission.csv', index = False)